In [1]:
import numpy as np
import emcee
import pandas as pd
import rebound
import os
from scipy.stats import norm, ks_2samp
import dask.dataframe as dd

In [2]:
if rebound.__githash__ != '6fb912f615ca542b670ab591375191d1ed914672':
    print('Check out rebound commit 6fb912f615ca542b670ab591375191d1ed914672 and rerun script if needed'.format(dataset))

In [3]:
seed = 0
np.random.seed(seed)
nwalkers = 20
ndim = 2
iterations = 1000

def lnprob(p, vec):
    diff = vec-p[0]
    N = len(vec)

    if p[1] <=0:
        return -np.inf
    try:
        probs = -0.5 * N * np.log(2. * np.pi) - N/2. * np.log(np.abs(p[1])**2) - 0.5 \
                                    * np.sum(( (vec - p[0]) / p[1] ) ** 2)
    except:
        probs = 0.00
    return probs
       
def log_prob_normed(mu, sigma, info):
    prob = -np.log(2*np.pi)/2. - np.log(sigma**2.)/2.-(1./(sigma**2.)/2./info.shape[0])*np.nansum((info-mu)**2.)
    return prob

def collision(reb_sim, col):
    reb_sim.contents._status = 5
    return 0

def run(row):
    tmax = 1e7
    ID = int(row['ID'])
    
    systemdir = distpath+'Res_sys_{0}_1e8/'.format(ID)
    for file in os.listdir(systemdir):
        if 'csv' in file:
            data = pd.read_csv(systemdir+file, index_col=0)
            data = data.apply(get_times, args=(systemdir,), axis=1)
            data.to_csv(csvpath+'Res_sys_{0}_{1}.csv'.format(ID, data.shape[0]))
            
    realization = data.loc[0]
    row['instability_time'] = realization['t']
    file = distpath+"Res_sys_{0}_1e8/simulation_archives/sa".format(ID)+realization['runstring']
    
    data = data[data["t"]<1e8]
    data = np.log10(data["t"].values)
    
    p0 = [np.random.rand(ndim) for i in range(nwalkers)]
    sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=[data], a=5)
    
    # Run 200 steps as a burn-in.
    pos, prob, state = sampler.run_mcmc(p0, 200)
    sampler.reset()
    pos, prob, state = sampler.run_mcmc(pos, iterations, rstate0=seed)
    
    maxprob_indice = np.argmax(prob)
    mean_fit, sigma_fit = pos[maxprob_indice]
    sigma_fit = np.abs(sigma_fit) 
    row['Mean'] = mean_fit
    row['Sigma'] = sigma_fit
    
    test = np.random.normal(loc=row['Mean'], scale=row['Sigma'], size = data.shape[0])

    try:
        statistic, KSpval = ks_2samp(data, test)
    except:
        statistic, KSpval = 0,0
        
    row['KSpval'] = KSpval
    
    sim = rebound.SimulationArchive(file)[0]
    sim.ri_whfast.keep_unsynchronized = 1
    sim.collision_resolve=collision
    sim.init_megno(seed=0)

    Nout = 1000
    times = np.logspace(0, np.log10(tmax), Nout)
    P0 = sim.particles[1].P

    try:
        sim.integrate(row['instability_time']/10, exact_finish_time=0)
        row['tlyap10'] = 1/sim.calculate_lyapunov()/P0
        if row['tlyap10'] < 0 or row['tlyap10'] > sim.t:
            row['tlyap10'] = sim.t
        row['Nlyap10'] = row['instability_time']  / row['tlyap10']
    except:
        row['tlyap10'] = np.nan
        row['Nlyap10'] = np.nan
    
    return row

def get_times(row, args):
    systemdir = args
    fcpath = systemdir+"/simulation_archives/sa"
    try:
        sa = rebound.SimulationArchive(fcpath + row["runstring"])
        row['t'] = sa[-1].t
        del sa
    except Exception as e:
        row['t'] = np.nan
    return row

In [23]:
csv_path = "../csvs/resonant/resonant_features/"
distpath = '../hussain2019data/resonant_distributions/'
for root, dirs, files in os.walk(distpath):
    planet_systems = dirs
    break
systems = np.array([s[:-4] for s in planet_systems])

In [76]:
%%time
for system in systems:
    systemdir = distpath+system+'_1e8/'
    for file in os.listdir(systemdir):
        if 'csv' in file:
            df = pd.read_csv(systemdir+file, index_col=0)
            df = df.apply(get_times, axis=1)
            df.to_csv(file_path+system+'_{0}.csv'.format(df.shape[0]))

Cannot read binary file. Check filename and file contents.
Cannot read binary file. Check filename and file contents.
Cannot read binary file. Check filename and file contents.
Cannot read binary file. Check filename and file contents.
Cannot read binary file. Check filename and file contents.
Cannot read binary file. Check filename and file contents.
Cannot read binary file. Check filename and file contents.
Cannot read binary file. Check filename and file contents.
Cannot read binary file. Check filename and file contents.
Cannot read binary file. Check filename and file contents.
Cannot read binary file. Check filename and file contents.
Cannot read binary file. Check filename and file contents.
Cannot read binary file. Check filename and file contents.
Cannot read binary file. Check filename and file contents.
Cannot read binary file. Check filename and file contents.
Cannot read binary file. Check filename and file contents.
Cannot read binary file. Check filename and file content

In [20]:
csvpath = "../csvs/resonant/resonant_features/"
distpath = '../hussain2019data/resonant_distributions/'
for root, dirs, files in os.walk(distpath):
    planet_systems = dirs
    break
    
df = pd.DataFrame([s.split("_")[-2] for s in planet_systems], columns=["ID"])
df = df.sort_values("ID")
df = df.reset_index(drop=True)
df.tail()

,ID
312,95
313,96
314,97
315,98
316,99


In [22]:
planet_systems

['Res_sys_2_1e8',
 'Res_sys_183_1e8',
 'Res_sys_83_1e8',
 'Res_sys_85_1e8',
 'Res_sys_286_1e8',
 'Res_sys_234_1e8',
 'Res_sys_103_1e8',
 'Res_sys_334_1e8',
 'Res_sys_76_1e8',
 'Res_sys_54_1e8',
 'Res_sys_243_1e8',
 'Res_sys_150_1e8',
 'Res_sys_386_1e8',
 'Res_sys_278_1e8',
 'Res_sys_352_1e8',
 'Res_sys_56_1e8',
 'Res_sys_60_1e8',
 'Res_sys_127_1e8',
 'Res_sys_92_1e8',
 'Res_sys_319_1e8',
 'Res_sys_138_1e8',
 'Res_sys_26_1e8',
 'Res_sys_279_1e8',
 'Res_sys_181_1e8',
 'Res_sys_266_1e8',
 'Res_sys_294_1e8',
 'Res_sys_339_1e8',
 'Res_sys_377_1e8',
 'Res_sys_66_1e8',
 'Res_sys_40_1e8',
 'Res_sys_116_1e8',
 'Res_sys_387_1e8',
 'Res_sys_309_1e8',
 'Res_sys_106_1e8',
 'Res_sys_88_1e8',
 'Res_sys_210_1e8',
 'Res_sys_173_1e8',
 'Res_sys_361_1e8',
 'Res_sys_45_1e8',
 'Res_sys_49_1e8',
 'Res_sys_213_1e8',
 'Res_sys_23_1e8',
 'Res_sys_78_1e8',
 'Res_sys_64_1e8',
 'Res_sys_82_1e8',
 'Res_sys_135_1e8',
 'Res_sys_84_1e8',
 'Res_sys_11_1e8',
 'Res_sys_21_1e8',
 'Res_sys_74_1e8',
 'Res_sys_310_1e8',
 'R

In [ ]:
%%time
ddf = dd.from_pandas(df, npartitions=24)
testres = run(df.iloc[0])
df = ddf.apply(run, axis=1, meta=pd.DataFrame([testres])).compute(scheduler='processes')

/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:89: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:138: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:138: RuntimeWarning: Encountered unkown field in file. File might have been saved with a different version of REBOUND.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:89: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/work

/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:138: RuntimeWarning: Encountered unkown field in file. File might have been saved with a different version of REBOUND.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:89: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:138: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:138: RuntimeWarning: Encountered unkown field in file. File might have been saved with a different version of REBOUND.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton

/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:138: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)
/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:138: RuntimeWarning: Encountered unkown field in file. File might have been saved with a different version of REBOUND.
  warnings.warn(message, RuntimeWarning)


In [45]:
df.to_csv('../csvs/resonant_summary.csv')

In [4]:
trappistdistpath = '../hussain2019data/trappist/simulation_archives/'
for root, dirs, files in os.walk(trappistdistpath):
    binaries = files
    break
binaries

['sarunstring98.bin',
 'sarunstring160.bin',
 'sarunstring111.bin',
 'sarunstring204.bin',
 'sarunstring415.bin',
 'sarunstring75.bin',
 'sarunstring445.bin',
 'sarunstring66.bin',
 'sarunstring262.bin',
 'sarunstring114.bin',
 'sarunstring168.bin',
 'sarunstring388.bin',
 'sarunstring383.bin',
 'sarunstring323.bin',
 'sarunstring137.bin',
 'sarunstring255.bin',
 'sarunstring392.bin',
 'sarunstring67.bin',
 'sarunstring100.bin',
 'sarunstring117.bin',
 'sarunstring329.bin',
 'sarunstring353.bin',
 'sarunstring418.bin',
 'sarunstring326.bin',
 'sarunstring453.bin',
 'sarunstring44.bin',
 'sarunstring438.bin',
 'sarunstring331.bin',
 'sarunstring393.bin',
 'sarunstring338.bin',
 'sarunstring154.bin',
 'sarunstring192.bin',
 'sarunstring145.bin',
 'sarunstring355.bin',
 'sarunstring182.bin',
 'sarunstring328.bin',
 'sarunstring40.bin',
 'sarunstring175.bin',
 'sarunstring5.bin',
 'sarunstring25.bin',
 'sarunstring128.bin',
 'sarunstring474.bin',
 'sarunstring247.bin',
 'sarunstring455.bin

In [10]:
%%time
def final_time(filename):
    try:
        sa = rebound.SimulationArchive(trappistdistpath + filename)
        sim = sa[0]
        P0 = sim.particles[1].P
        return sa[-1].t/P0
    except Exception as e:
        print(e, filename)
        return np.nan
    
trappisttimes = [final_time(f) for f in binaries]

CPU times: user 64.8 ms, sys: 24.5 ms, total: 89.3 ms
Wall time: 87.9 ms


/mnt/ssd/Dropbox/Dropbox (Princeton)/workspace/rebound/rebound/simulationarchive.py:89: RuntimeWarning: Binary file was saved with a different version of REBOUND. Binary format might have changed.
  warnings.warn(message, RuntimeWarning)


In [11]:
trappisttimes

[263050.1241269671,
 533679.3215816992,
 538121.4776589728,
 300209.1264437978,
 437795.2404610296,
 648305.8900830419,
 2195944.0669444255,
 1064352.7739282392,
 3288519.0796874715,
 738594.0269801947,
 1700806.6188115038,
 348883.0796523445,
 520607.67266538297,
 659278.0855687605,
 4666981.669802667,
 2414875.2882287903,
 416204.672898046,
 65269.18234929589,
 28251933.77867322,
 803119.5361722411,
 291169.17223194643,
 832866.6428479806,
 273027.1524663119,
 389442.3748310469,
 899061.0522797046,
 103551.76605486857,
 171502.54409269185,
 559824.8017983551,
 3632379.141552788,
 3916391.271029597,
 151364.54361832052,
 711674.250768411,
 1164962.6166694693,
 19003763.02525187,
 20469.168597134398,
 873609.1544903236,
 1202045.8589663259,
 780645.2415752906,
 560288.4371597898,
 357858.7747792637,
 142593.403691684,
 596653.6407072715,
 305678.56184048543,
 4810214.064785825,
 421659.7322730372,
 932602.7012340012,
 103247.20224075974,
 475656.4161749358,
 606156.0178736777,
 168313.

In [19]:
trap = pd.DataFrame(np.array([binaries, trappisttimes]).T, columns=['runstring', 't'])
trap.to_csv('../csvs/trappist.csv')